In [1]:
import numpy as np
import math
import timeit
from random import shuffle
from statistics import median

In [2]:
def _closest_pow2(n):
    return 2**(math.ceil(math.log(n, 2)))

def _fft_rec(t,K):
    ret = []
    
    if K == 0:
        return t
    else:
        evens, odds = t[::2],t[1::2]
        rec_evens = _fft_rec(evens,K-1)
        rec_odds = _fft_rec(odds,K-1)
        a_len = int(2**(K-1))
        
        for i in range(len(t)):
            n =  rec_evens[i % a_len] + np.exp((2j*math.pi * i/2**K)) * rec_odds[i % a_len]
            ret.append(np.around(n, 3))
        
        return ret
    
def fft(t):
    t_len = len(t)
    
    next_pow = _closest_pow2(t_len)
    k = math.log(next_pow,2)
    num_add_zeros = next_pow - t_len
    zeros_array = np.array([0]*num_add_zeros)
    complete_array = np.concatenate((t,zeros_array),axis=0)
    

    return np.array(_fft_rec(complete_array,k))


def invert_fft(t, fft_func=fft):
    t_len = len(t)
    
    next_pow = _closest_pow2(t_len)
    k = math.log(next_pow,2)
    num_add_zeros = next_pow - t_len
    zeros_array = np.array([0]*num_add_zeros)
    complete_array = np.concatenate((t,zeros_array),axis=0)
    
    t_conj = [np.conj(n) for n in complete_array]
    
    dft = fft_func(t_conj)
    
    dft_conj = [np.conj(n) for n in dft]
    
    return [n/len(t) for n in dft_conj]

In [3]:
a = np.array([1,2,3,4])

s = fft(a)
#t = invert_fft(a)

print(s)


[10.+0.j -2.-2.j -2.+0.j -2.+2.j]


In [4]:
b = list(np.random.randint(low=0, high=5, size=15))
b.append(10)
b

[4, 3, 0, 3, 2, 2, 3, 3, 2, 2, 3, 0, 1, 4, 2, 10]

In [5]:
def rand_polinomio(long=2**10,base=10):
    ret = list(np.random.randint(low=0, high=base, size=long-1).astype('uint8'))
    #print(ret)
    ret.append(np.random.randint(low=1, high=base))
    return ret

def poli_2_num(l_pol,base=10):
    ret = 0
    
    for i in range(len(l_pol)):
        ret += int(l_pol[i]*pow(base, i))
    return ret


l = rand_polinomio()
#print(l)
print(poli_2_num(l, 2))

603482997276878112577658338409925746546124423773217985973160956811012130085706364754221648378707746158984076276277270775665954232987949822394617411609513664485156652240167055483147638627586376713861455534541453850394597622788031367520133215377311862981897807260841533021075476251529436908774915074975378683373


In [6]:
def rand_numero(num_digits,base=10):
    poli = rand_polinomio(num_digits, base)
    
    return poli_2_num(poli, base)

def num_2_poli(num,base=10):
        
    poli = []
    while True:
        remain = num % base
        poli.append(remain)
        num = math.floor(num / base)
        if num == 0:
            break

    return poli


print(rand_numero(5))
print(num_2_poli(1801, 5))

57073
[1, 0, 2, 4, 2]


In [7]:
def _padding_polinomios(l_pol_1, l_pol_2):
    long_prod = len(l_pol_1)+len(l_pol_2)-1 #longitud del polinomio producto
    
    next_pow2 = _closest_pow2(long_prod)
    
    num_add_zeros_pol1 = next_pow2 - len(l_pol_1)
    zeros_array_pol1 = np.array([0]*num_add_zeros_pol1)
    complete_array_pol1 = np.concatenate((l_pol_1,zeros_array_pol1),axis=0)
    
    num_add_zeros_pol2 = next_pow2 - len(l_pol_2)
    zeros_array_pol2 = np.array([0]*num_add_zeros_pol2)
    complete_array_pol2 = np.concatenate((l_pol_2,zeros_array_pol2),axis=0)
    
    return complete_array_pol1, complete_array_pol2

def mult_polinomios(l_pol_1, l_pol_2):
    prod = [0]*(len(l_pol_1)+len(l_pol_2)-1)
    
    for i in range(len(l_pol_1)):
        for j in range(len(l_pol_2)):
            prod[i+j] = prod[i+j] + l_pol_1[i] * l_pol_2[j]
    
    return prod

def mult_polinomios_fft(l_pol_1, l_pol_2, fft_func=fft):
    ret = []
    
    l_pol_1_pad, l_pol_2_pad = _padding_polinomios(l_pol_1, l_pol_2)
    
    
    # Realizamos la fft para pol_1 y pol_2
    fft_pol_1 = fft_func(l_pol_1_pad)
    fft_pol_2 = fft_func(l_pol_2_pad)
    
    
    
    if len(fft_pol_1) != len(fft_pol_2):
        pass
    else:
        for (i,j) in zip(fft_pol_1,fft_pol_2):
            ret.append(i*j)
    
    prod_pol = [n.real for n in invert_fft(ret)]
    
    
    return prod_pol


def mult_numeros(num1, num2):
    pol1 = num_2_poli(num1)
    pol2 = num_2_poli(num2)
    
    pol_mul = mult_polinomios(pol1, pol2)
    
    return poli_2_num(pol_mul)


def mult_numeros_fft(num1, num2, fft_func=fft):
    pol1 = num_2_poli(num1)
    pol2 = num_2_poli(num2)
    
    
    pol_mul = mult_polinomios_fft(pol1, pol2, fft_func)
    
    return poli_2_num(pol_mul)

l_a = [1,2,5]
l_b = [1,2,3]
print(mult_polinomios(l_a, l_b))
print(mult_polinomios_fft(l_a,l_b))
print(mult_numeros(304, 509))
print(mult_numeros_fft(304, 509))

[1, 4, 12, 16, 15]
[1.0, 3.99975, 11.9995, 15.99875, 15.0, 0.00025, 0.0005, 0.00125]
154736
154736


In [8]:
def time_mult_numeros(n_pairs, num_digits_ini, num_digits_fin, step):
    
    def local_rand_numeros(num):
        return mult_numeros(rand_numero(num), rand_numero(num))
    
    num_pair = 0
    times = []
    
    while num_pair < n_pairs:
        num_digits_actual = num_digits_ini
    
        while num_digits_actual <= num_digits_fin:
            # , setup = "from __main__ import mult_numeros, rand_numero, num_digits_actual", number = 1
            times.append(timeit.timeit(lambda: local_rand_numeros(num_digits_actual), number=1))
            num_digits_actual += step
        
        num_pair += 1
    
    return np.array(times)


def time_mult_numeros_fft(n_pairs, num_digits_ini, num_digits_fin, step, fft_func=fft):
    
    def local_rand_numeros(num):
        return mult_numeros_fft(rand_numero(num), rand_numero(num),fft_func=fft_func)
    
    num_pair = 0
    times = []

    while num_pair < n_pairs:
        num_digits_actual = num_digits_ini
    
        while num_digits_actual <= num_digits_fin:
            # , setup = "from __main__ import mult_numeros, rand_numero, num_digits_actual", number = 1
            times.append(timeit.timeit(lambda: local_rand_numeros(num_digits_actual), number=1))
            num_digits_actual += step
        
        num_pair += 1
    
    return np.array(times)


print(time_mult_numeros(3, 3, 6, 1))
print('\n')
print(time_mult_numeros_fft(3, 3, 6, 1))

[2.27199998e-04 6.45129912e-05 6.83979888e-05 7.28059968e-05
 5.29370009e-05 5.84689988e-05 6.35280012e-05 7.05019920e-05
 5.23089984e-05 5.71099954e-05 6.30259892e-05 7.06650026e-05]


[0.0010095  0.00196675 0.00358425 0.00226229 0.00188056 0.00151102
 0.00585643 0.00311762 0.00096086 0.00094397 0.00219247 0.00219811]


In [9]:
def split(t, ini, fin):
    
    assert ini >= 0 and fin < len(t), "Los indices 'ini' y/o 'fin' se han introducido incorrectamente"
    
    pivot = t[ini]
    
    leftPointer, rightPointer = ini+1, fin
    
    while True:
        
        while leftPointer < fin and t[leftPointer] < pivot:
            leftPointer += 1
        
        while rightPointer > ini and t[rightPointer] > pivot:
            rightPointer -= 1
        
        if leftPointer >= rightPointer:
            break;
        else:
            t[leftPointer], t[rightPointer] = t[rightPointer], t[leftPointer]
    
    t[ini], t[rightPointer] = t[rightPointer], t[ini]
    
    return rightPointer


def split_pivot(t, ini, fin, pivot=None):
    
    if (pivot == None):
        return split(t, ini, fin)
    
    assert ini >= 0 and fin < len(t), "Los indices 'ini' y/o 'fin' se han introducido incorrectamente"
    assert pivot in t, "pivot no está en la lista"
    
    pivotIndex = t.index(pivot)
    
    t[pivotIndex], t[fin] = t[fin], t[pivotIndex]  # Movemos el pivote al final
    index = ini
    for i in range(ini, fin):
        if t[i] < pivot:
            t[index], t[i] = t[i], t[index]
            index += 1
    t[fin], t[index] = t[index], t[fin]  # Movemos el pivote a su respectivo lugar
    
    return index



def qselect(t, ini, fin, ind, pivot=None):
    
    if ini == fin:
        return t[ini]
    
    split_p = split_pivot(t, ini, fin, pivot)
    
    l = split_p - ini + 1
    
    if l == ind:
        return t[split_p]
    elif ind < l:
        return qselect(t, ini, split_p-1, ind, pivot)
    else:
        return qselect(t, split_p+1, fin, ind -l, pivot)
    

def qselect_sr(t, ini, fin, ind, pivot=None):
    
    if ini == fin:
        return t[ini]
    
    while True:
    
        split_p = split_pivot(t, ini, fin, pivot)

        l = split_p - ini + 1

        if l == ind:
            return t[split_p]
        elif ind < l:
            fin = split_p-1
        else:
            ini = split_p+1
            ind -= l


                
def pivot_5(t, ini, fin):
    t = t[ini:fin+1]
    
    mid_lists = [t[i:i+5] for i in range(0, len(t), 5)]
    medians = [math.ceil(median(x)) for x in mid_lists]
    
    return math.ceil(median(medians))
    
    

def qselect_5(t, ini, fin, pos):
    
    if ini == fin:
        return t[ini]
    
    while True:
        
        pivotIndex = pivot_5(t, ini, fin)
        pivotIndex = split_pivot(t, ini, fin, pivotIndex)

        l = pivotIndex - ini + 1

        if l == pos:
            return t[pivotIndex]
        elif pos < l:
            fin = pivotIndex-1
        else:
            ini = pivotIndex+1
            pos -= l
    '''while True:
        if ini == fin:
            return ini
            
        pivotIndex = pivot_5(t, ini, fin)
        pivotIndex = split_pivot(t, ini, fin, pivotIndex)
        if pos == pivotIndex:
            return pos
        elif pos < pivotIndex:
            fin = pivotIndex - 1
        else:
            ini = pivotIndex + 1'''
        #return qselect_sr(t, ini, fin, pos)


#import random
#random.seed(2)
a = [n for n in range(10)]
shuffle(a)
#print(split(a, 0, 5))
print(a)
#print(split_pivot(a, 0, 5))
#print(a)
#print(qselect_sr(a, 0, 9, 3))
#print(pivot_5(a, 0, 9))
print(qselect_5(a,0,9,3))

[4, 6, 7, 1, 2, 3, 8, 0, 9, 5]
2


In [10]:
def qsort_5(t, ini, fin):
    
    if ini < fin:
        
        pivotIndex = pivot_5(t, ini, fin)
        pivotIndex = split_pivot(t, ini, fin, pivotIndex)
        
        qsort_5(t, ini, fin-1)
        qsort_5(t, ini+1, fin)


a = [n for n in range(10)]
shuffle(a)
print(a)
qsort_5(a, 0, 9)
print(a)

[3, 4, 6, 9, 7, 1, 5, 2, 0, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [1]:
print(1. == 1)

True
